# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [20]:
#Your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style('darkgrid')

import warnings 
warnings.filterwarnings('ignore')

df = pd.read_csv('homepage_actions.csv')
df

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [7]:
df.action.value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [12]:
# create subset of those who viewed and those who clicked with unique id 

viewed = set(df[df.action == "view"]['id'].unique())
clicked = set(df[df.action == "click"]['id'].unique())

print("Number of viewers:", len(viewed))
print("Number who clicked through:", len(clicked))
print("Number who viewed but did not click", len(viewed) - len(clicked))
print("Percentage who clicked:", round(len(clicked)/len(viewed)*100))          

Number of viewers: 6328
Number who clicked through: 1860
Number who viewed but did not click 4468
Percentage who clicked: 29


In [19]:
#create subsets of the experimental group and control group

control = set(df[df.group == 'control']['id'].unique())
exp = set(df[df.group == 'experiment']['id'].unique())

print("Control group size:", len(control))
print("Experimental group size:", len(exp))
print("Combined group size:", len(control) + len(exp))
print("There is no overlap in the control and experimental groups")

Control group size: 3332
Experimental group size: 2996
Combined group size: 6328
There is no overlap in the control and experimental groups


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [32]:
#Your code here


# Sum all the views and click throughs by control and experiment 
control_views = len(df[(df.action == 'view') & (df.group == "control")])
control_click = len(df[(df.action == 'click') & (df.group == "control")])

experiment_views = len(df[(df.action == 'view') & (df.group == "experiment")])
experiment_click = len(df[(df.action == 'click') & (df.group == "experiment")])


print("Control views:", control_views)
print("Control click throughs:", control_click)
print("Experimental views", experiment_views)
print("Experimental click throughs", experiment_click)

      
      
      

control views: 3332
control click throughs: 932
Experimental views 2996
Experimental click throughs 928


In [33]:
#set up for a chi-square test
observations = np.array([experiment_click, experiment_views - experiment_click])

expectations = np.array([control_click, control_views- control_click])

print('OBSERVED (expermiment):', observations)
print('EXPECTED (control):', expectations)

OBSERVED (expermiment): [ 928 2068]
EXPECTED (control): [ 932 2400]


In [35]:
import scipy.stats as stats
stats.chisquare(f_obs=observations, f_exp=expectations)


Power_divergenceResult(statistic=45.943834048640916, pvalue=1.2169234221091101e-11)

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [36]:
#Click through rates for each group

experiment_percent = experiment_click/experiment_views*100
print(f'Percent Experiment Converted: {experiment_percent:.5}%')
control_percent = control_click/control_views*100
print(f'Percent Control Converted: {control_percent:.5}%')

print(f'Difference between experiment & control {experiment_percent-control_percent:.3}%')



Percent Experiment Converted: 30.975%
Percent Control Converted: 27.971%
Difference between experiment & control 3.0%


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [43]:
print("Expected click throughs of experimental group:", round(control_percent * len(exp)/100))

print("Actual click through of exp. group", experiment_click)

Expected click throughs of experimental group: 838
Actual click through of exp. group 928


In [59]:
#Your code here
expected_exp_clicks = round(control_percent * len(exp)/100)
expected_exp_clicks



838

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [ ]:
#Your code here

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Because I performed a chi-squared test and took a different approach to this business question it was more difficult to verify my results. There does appear to be a statisitically significant results between the experimental and control groups with a p-value far below that of our standard alpha. There is a 3% higher conversion rate in our experimentral group. Whether that is enough for the business stakeholder to make changes could be up for further discussion and analysis. 

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.